In [1]:
import pandas as pd
import os


# Checking the current working directory
print(os.getcwd())

c:\Users\srian\OneDrive\Documents\GitHub\Generative-AI-project


In [2]:
# loading the pump data
pump_data = pd.read_excel("Input Data/Input Parameters V3_Final.xlsx")

pump_data

,H0,H1,H2,M0,M1,M2,Q0,Q1,Q2,w0,w1,w2
0,13.816539,12.683127,15.353757,13.438797,12.334016,15.213747,15.213747,40.616147,44.606094,2901.105151,2903.029505,2900.430316
1,20.281990,18.766565,22.252293,15.872541,14.594791,17.554310,36.247222,34.817690,37.910103,2901.125515,2925.276252,2904.637849
2,24.123035,22.533191,26.194417,16.568131,15.315073,18.296685,32.261669,31.174444,33.633477,2901.127056,2903.454979,2887.387053
3,26.682970,24.943127,28.720053,16.665619,15.364651,18.105263,29.392805,28.398185,30.479075,2901.125443,2894.047578,2896.718011
4,28.535620,26.816259,30.670728,16.528010,15.334138,17.895941,27.185814,26.343184,28.182353,2901.124477,2895.211990,2925.546931
...,...,...,...,...,...,...,...,...,...,...,...,...
1007,40.555509,38.436045,42.408154,16.630446,15.337242,18.255629,36.103472,40.427373,44.484408,2901.126850,2924.717795,2925.358775
1008,40.603659,38.541118,42.488757,16.657502,15.347829,18.265498,36.107633,40.469671,44.509301,2901.126915,2924.773016,2925.363523
1009,40.607392,38.544605,42.501008,16.662002,15.352638,18.271743,36.181910,40.478691,44.552292,2901.126972,2925.035387,2925.365758
1010,40.622912,38.587633,42.509359,16.662066,15.354606,18.281483,36.190498,40.571086,44.579967,2901.126976,2925.083548,2925.439889


In [3]:
# Data Preprocessing

from sklearn.preprocessing import StandardScaler

# Handling the missing values
pump_data.fillna(method='ffill', inplace=True)

# Normalizing the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(pump_data)

C:\Users\srian\AppData\Local\Temp\ipykernel_22988\892942249.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pump_data.fillna(method='ffill', inplace=True)


In [4]:
# Feature Engineering
# Extract features from the data (example for vibration data):

import numpy as np

def extract_features(data):
    features = []
    for col in data.columns:
        series = data[col]
        features.append(np.mean(series))
        features.append(np.std(series))
        features.append(np.max(series))
        features.append(np.min(series))
    return features

features = extract_features(pump_data)

In [6]:
# Model Selection
# Using Variational Autoencoders (VAEs) as an example for a generative model:

import tensorflow as tf
from tensorflow.keras import layers

# Encoder
input_dim = data_scaled.shape[1]
latent_dim = 2

inputs = tf.keras.Input(shape=(input_dim,))
h = layers.Dense(128, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim, name='z_mean')(h)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Decoder
decoder_h = layers.Dense(128, activation='relu')
decoder_mean = layers.Dense(input_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# VAE model
vae = tf.keras.Model(inputs, x_decoded_mean)

# Loss function
reconstruction_loss = tf.keras.losses.mse(inputs, x_decoded_mean)
reconstruction_loss *= input_dim
kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

ModuleNotFoundError: No module named 'tensorflow.python'